In [1]:
import sys
sys.path.append("/opt/projects/aoi/led_detection/training/scratch/")

In [1]:
import os
import numpy as np
import onnxruntime as ort
from PIL import Image
import torch
import torch.nn as nn
import onnx

# 🚀 **Define Normalizer**
class Normalizer:
    def __init__(self):
        self.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1, 1, 3)
        self.std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1, 1, 3)

    def __call__(self, img):
        return (img - self.mean) / self.std

# **ONNX Wrapper Class**
class ONNXWrapper(nn.Module):
    def __init__(self, model):
        super(ONNXWrapper, self).__init__()
        self.model = model
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        print(f"✅ ONNXWrapper Forward running, input shape: {x.shape}")
        scores, labels, boxes = self.model(x)


        # **Apply Sigmoid normalization to scores**
        scores = self.sigmoid(scores).to(dtype=torch.float32)
        labels = labels.to(dtype=torch.float32)  # ✅ 改成 int32
        boxes = boxes.to(dtype=torch.float32)

        
        boxes = boxes.unsqueeze(0) if boxes.dim() == 2 else boxes  # (18, 4) → (1, 18, 4)
        scores = scores.unsqueeze(0)  # (18,) → (1, 18)
        labels = labels.unsqueeze(0)  # (18,) → (1, 18)

        # **Prevent empty outputs from affecting ONNX inference**
        if scores.numel() == 0 or labels.numel() == 0 or boxes.numel() == 0:
            print("⚠️ No detections found, filling with default values")
            scores = torch.tensor([[0.01]], dtype=torch.float32, device=x.device)
            labels = torch.tensor([[0]], dtype=torch.int64, device=x.device)
            boxes = torch.tensor([[[0.0, 0.0, 1.0, 1.0]]], dtype=torch.float32, device=x.device)



        """        # **Ensure boxes have the correct dimension (batch_size, num_detections, 4)**
        if boxes.dim() == 2:
            boxes = boxes.unsqueeze(1)  # Add an extra dimension
        elif boxes.dim() == 3:
            assert boxes.shape[2] == 4, "Boxes must have 4 coordinate values"
        
        # **Adjust box dimensions**
        if boxes.shape[1] == 1:
            boxes = boxes.squeeze(1)  # Remove unnecessary dimension"""

        print(f"✅ ONNXWrapper Output: scores {scores.shape}, labels {labels.shape}, boxes {boxes.shape}")
        print(f"📦 PyTorch Predicted boxes: {boxes}")
        return scores, labels, boxes


# **Load PyTorch model**
model_path = "/home/guoy/led_detection/training/RetinaNet/RetinaNet_model_final.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device)

# **Unwrap DataParallel**
if isinstance(model, torch.nn.DataParallel):
    print("⚠️ Detected DataParallel, unwrapping the model...")
    model = model.module

# **Wrap the forward() function**
wrapped_model = ONNXWrapper(model).to(device)
wrapped_model.eval()

# **🚀 Read and normalize a real image**
image_path = "/home/guoy/led_detection/training/RetinaNet/000002_2024_10_28_15_40_10_540_0.jpg"
image = Image.open(image_path).convert("RGB")
img_np = np.array(image).astype(np.float32) / 255.0
img_np = Normalizer()(img_np)
img_np = np.transpose(img_np, (2, 0, 1))

# **Create PyTorch input tensor**
input_tensor = torch.tensor(img_np, dtype=torch.float32).unsqueeze(0).to(device)
print(f"✅ Real image input shape: {input_tensor.shape}")

# **🚀 Test PyTorch forward() first**
wrapped_model(input_tensor)

# **Export to ONNX**
onnx_path = "/home/guoy/led_detection/training/RetinaNet/RetinaNet_v1.onnx"
torch.onnx.export(
    wrapped_model,
    input_tensor,
    onnx_path,
    opset_version=11,
    input_names=["input"],
    output_names=["scores", "labels", "boxes"],
    dynamic_axes={
        "scores": {0: "batch_size", 1: "num_detections"},
        "labels": {0: "batch_size", 1: "num_detections"},
        "boxes": {0: "batch_size", 1: "num_detections", 2: "coords"}  # coords represent bounding box coordinates
    },
    do_constant_folding=True 
)

print(f"✅ Successfully exported ONNX: {onnx_path}")

try:
    ort_session = ort.InferenceSession(onnx_path)
    
    # Get input name
    input_name = ort_session.get_inputs()[0].name
    
    # Ensure the input data has the correct shape
    onnx_input = input_tensor.detach().cpu().numpy()
    
    # Perform ONNX inference
    outputs = ort_session.run(None, {input_name: onnx_input})
    
    # Ensure the output shapes match expectations
    print(f"✅ ONNX Inference scores.shape={outputs[0].shape}")
    print(f"✅ ONNX Inference labels.shape={outputs[1].shape}")
    print(f"✅ ONNX Inference boxes.shape={outputs[2].shape}")
    
except Exception as e:
    print(f"❌ ONNX Inference failed: {e}")


/tmp/ipykernel_380511/2875996582.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


⚠️ Detected DataParallel, unwrapping the model...
✅ Real image input shape: torch.Size([1, 3, 640, 640])
✅ ONNXWrapper Forward running, input shape: torch.Size([1, 3, 640, 640])
✅ ONNXWrapper Output: scores torch.Size([1, 18]), labels torch.Size([1, 18]), boxes torch.Size([1, 18, 4])
📦 PyTorch Predicted boxes: tensor([[[ 39.9766, 447.2587,  81.3504, 480.3082],
         [ 52.9102, 221.9566,  92.4365, 256.0235],
         [439.2004, 467.7859, 481.3566, 503.3602],
         [172.5506, 452.7335, 214.7695, 488.5393],
         [508.3899, 471.9164, 550.7650, 507.0966],
         [183.8363, 228.0184, 227.6414, 263.4944],
         [118.2582, 223.6453, 162.2453, 259.8027],
         [105.3479, 449.3438, 148.9051, 485.3590],
         [456.8612, 244.0427, 493.2340, 276.1395],
         [252.0436, 231.6617, 293.3106, 266.3990],
         [305.9417, 460.1208, 347.8703, 495.4235],
         [374.4185, 464.6841, 413.6945, 498.9632],
         [321.7256, 236.2244, 358.1698, 268.5798],
         [523.9587, 247.5

/home/guoy/led_detection/training/RetinaNet/retinanet/anchors.py:24: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  image_shape = np.array(image_shape)
/home/guoy/led_detection/training/RetinaNet/retinanet/anchors.py:38: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.from_numpy(all_anchors.astype(np.float32)).cuda()
/home/guoy/led_detection/training/RetinaNet/retinanet/utils.py:138: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you

✅ Successfully exported ONNX: /home/guoy/led_detection/training/RetinaNet/RetinaNet_v1.onnx
✅ ONNX Inference scores.shape=(1, 18)
✅ ONNX Inference labels.shape=(1, 18)
✅ ONNX Inference boxes.shape=(1, 18, 4)
